In [57]:
import tensorflow as tf

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time


In [2]:
NPOB = 100
NGEN = 100
Pc = 0.6
Pm = 0.4
C = 0.4
NRES = 1
NSAMPLE = 1


In [31]:

# Read the data from the excel file (assuming it's loaded)
data = pd.read_excel('Datos TablaInversiones.xlsx')
data = data.dropna()
data = data.reset_index(drop=True)
# Rename columns
data.columns = ["Empresa", "Enero", "Febrero", "Marzo", "Abril", "Mayo", "Junio", "Julio", "Agosto", "Septiembre", "Octubre", "Noviembre", "Diciembre"]

data

,Empresa,Enero,Febrero,Marzo,Abril,Mayo,Junio,Julio,Agosto,Septiembre,Octubre,Noviembre,Diciembre
0,IBM,-0.012546,0.045071,0.023199,0.009866,-0.034398,-0.034401,-0.044192,0.036618,0.010112,0.020807,-0.047942,0.046991
1,Nvidia,0.033244,-0.028766,-0.031818,-0.03166,-0.019576,0.002476,-0.006805,-0.020877,0.011185,-0.036051,-0.020786,-0.013364
2,Kellogs,-0.004393,0.028518,-0.030033,0.001423,0.009241,-0.045355,0.010754,-0.032948,-0.043495,0.044889,0.046563,0.03084
3,Microsoft,-0.019539,-0.040233,0.018423,-0.005985,-0.037796,-0.000482,-0.046561,0.040932,-0.024122,0.016252,-0.018829,0.002007
4,BP,0.004671,-0.031515,0.046958,0.027513,0.04395,0.039483,0.00979,0.042187,-0.041151,-0.030402,-0.045477,-0.017467
5,UPS,-0.011132,-0.022865,0.032874,-0.014325,-0.021907,0.00427,-0.035908,0.03022,-0.042545,0.048689,0.027224,-0.030128
6,Amazon,-0.049448,0.031546,0.020686,0.022901,0.027127,-0.042596,-0.014153,-0.038413,0.03631,0.01233,-0.01691,-0.043644
7,Intel,-0.018902,-0.017482,0.022961,0.013756,0.038721,-0.002779,-0.038041,0.021324,0.026079,0.006128,0.027097,-0.00062
8,Ford,0.002273,-0.007246,-0.047458,-0.039211,-0.046857,0.013641,-0.018564,0.000857,0.040757,-0.025071,-0.008962,0.025555
9,Exxon,-0.02712,-0.042302,-0.021025,-0.033878,0.04297,0.030812,0.01334,0.037146,0.030367,-0.031343,0.039256,0.003934


In [4]:


def cromosoma():
    # Generate a vector of 20 random numbers between 0 and 1 that sum 1
    random_vector = tf.random.uniform((20,), minval=0.0, maxval=1.0, dtype=tf.float32)
    return random_vector / tf.reduce_sum(random_vector)


In [104]:
type(cromosoma())

tensorflow.python.framework.ops.EagerTensor

In [51]:
ri_cache = {}

@tf.function
def ri(company):
    if company in ri_cache:
        return ri_cache[company]
    else:
        # Calculate the return rate of the company
        r = tf.convert_to_tensor(data.loc[data["Empresa"] == company].iloc[0, 1:].sum()/12 , dtype=tf.float32)

        #add to cache
        ri_cache[company] = r
        
        # Calculate the risk rate of the company
        return r

In [103]:
type(ri("IBM"))

tensorflow.python.framework.ops.EagerTensor

In [42]:
type(data.loc[data["Empresa"] == "IBM"].iloc[0, 1:].values)

numpy.ndarray

In [98]:
oi_cache = {}

@tf.function
def calcularOi(i):
    if i not in oi_cache:
        suma = 0
        for m in range(12):
            suma += tf.square(data.iloc[i][m+1] - ri(data.iloc[i]["Empresa"]))

        oi_cache[i] = tf.sqrt(suma/11)
    return oi_cache[i]



In [99]:
print(calcularOi(1))

tf.Tensor(0.020550886, shape=(), dtype=float32)


/tmp/__autograph_generated_filerdybfyuh.py:31: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  suma += ag__.converted_call(tf.square, (data.iloc[i][m + 1] - ag__.converted_call(ri, (data.iloc[i]['Empresa'],), None, fscope),), None, fscope)


In [107]:

@tf.function
def calcularOc(cromosoma):
    """Calculates Oc using TensorFlow for efficient tensor operations.

    Args:
        cromosoma: A TensorFlow eager tensor representing the chromosome.

    Returns:
        A TensorFlow tensor representing the calculated Oc value.
    """

    # Calculate Oi for each element using map_fn with fn_output_signature
    oi_i = tf.map_fn(calcularOi, cromosoma, fn_output_signature=tf.float32)

    # Efficiently calculate the outer product using matmul
    outer_product = tf.linalg.matmul(cromosoma[:, tf.newaxis], oi_i[tf.newaxis, :])

    # Element-wise multiplication and summation with broadcasting
    resultado = tf.reduce_sum(outer_product * outer_product, axis=[1, 2]) * 0.25

    return tf.sqrt(resultado)


In [108]:
i = cromosoma()
calcularOc(i) 

TypeError: in user code:

    File "/tmp/ipykernel_4532/3468077854.py", line 13, in calcularOc  *
        oi_i = tf.map_fn(calcularOi, cromosoma, fn_output_signature=tf.float32)
    File "/tmp/ipykernel_4532/1711145092.py", line 5, in calcularOi  *
        if i not in oi_cache:

    TypeError: Tensor is unhashable. Instead, use tensor.ref() as the key.


In [ ]:


@tf.function
def ratioSharpe(cromosoma):
    # Function to calculate Sharpe ratio
    rc = calcularRc(cromosoma)
    desv = calcularOc(cromosoma)
    return (rc - 0.003757) / desv


@tf.function
def calcularRc(cromosoma):
    # Function to calculate expected return (vectorization)
    return tf.tensordot(cromosoma, ri(data["Empresa"]), 1)


def ruleta(fitness_gen, fitness_total, Pm):
    """
    Function that decides if to mutate a gene using the ruleta.

    Args:
      fitness_gen: Fitness of the gene to evaluate.
      fitness_total: Fitness total of the chromosome.
      Pm: Probability of individual mutation.

    Returns:
      True if the gene should mutate, False otherwise.
    """
    fitness_relativo = fitness_gen / fitness_total
    r = tf.random.uniform([], minval=0.0, maxval=1.0)

    # Apply ruleta
    return tf.greater(r, 1 - fitness_relativo * Pm)
